In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
pd.set_option('display.max_columns', None)  # this will output all columns when display a pandas dataframe

In [2]:
df=pd.read_csv('2012-2013_cleaned2.csv', index_col=0)

In [3]:
df

,loan_amnt,term,int_rate,installment,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,acc_open_past_24mths,mort_acc,percent_bc_gt_75,loan_status
0,12000.0,0.124432,7.62,373.94,32,Systems Engineer,3.0,0.142148,96500.0,0.122274,2013-12-01,Borrower added on 12/31/13 > Bought a new ho...,0.165022,Debt Consolidation and Credit Transfer,782xx,0.140666,12.61,0.0,123,705.0,709.0,0.0,17.0,13248.0,55.7,30.0,4.00000,1.00000,100.000000,0.0
1,27050.0,0.124432,10.99,885.46,28,Team Leadern Customer Ops & Systems,10.0,0.158888,55000.0,0.177538,2013-12-01,Borrower added on 12/31/13 > Combining high ...,0.165022,Debt Consolidation,481xx,0.165297,22.87,0.0,326,730.0,734.0,0.0,14.0,36638.0,61.2,27.0,3.00000,4.00000,25.000000,0.0
2,12000.0,0.124432,11.99,398.52,27,LTC,10.0,0.142148,130000.0,0.160987,2013-12-01,NaN,0.165022,Debt consolidation,809xx,0.126967,13.03,0.0,193,715.0,719.0,1.0,9.0,10805.0,67.0,19.0,4.00000,3.00000,1.000000,0.0
3,28000.0,0.124432,7.62,872.52,32,Area Sales Manager,5.0,0.142148,325000.0,0.160987,2013-12-01,NaN,0.165022,Pay off other Installment loan,945xx,0.148025,18.55,0.0,229,745.0,749.0,1.0,15.0,29581.0,54.6,31.0,6.00000,5.00000,16.700000,0.0
4,27600.0,0.255994,19.97,730.78,15,Street Operations Supervisor,6.0,0.142148,73000.0,0.160987,2013-12-01,Borrower added on 12/31/13 > I had some wate...,0.165022,Consolidation of debt and home improve.,802xx,0.126967,23.13,1.0,294,665.0,669.0,1.0,10.0,27003.0,82.8,24.0,2.00000,4.00000,80.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180304,12000.0,0.124432,7.90,375.49,31,mpmi,3.0,0.142148,42000.0,0.122274,2012-01-01,Borrower added on 12/11/11 > pay off credit ...,0.128642,credit card payoff,619xx,0.137417,9.83,0.0,129,720.0,724.0,0.0,9.0,14378.0,52.3,14.0,3.92953,1.81102,53.558127,0.0
180305,20500.0,0.124432,16.77,728.54,18,NaN,7.0,0.172892,60000.0,0.160987,2012-01-01,Borrower added on 12/08/11 > I will be payin...,0.165022,Payoff Loan,100xx,0.171307,16.40,1.0,279,700.0,704.0,3.0,10.0,15417.0,58.4,20.0,3.92953,1.81102,53.558127,0.0
180306,15000.0,0.124432,15.27,521.97,21,NaN,3.0,0.172892,57600.0,0.160987,2012-01-01,NaN,0.254734,business loan,900xx,0.148025,8.35,2.0,96,680.0,684.0,1.0,18.0,8897.0,33.1,30.0,3.92953,1.81102,53.558127,0.0
180307,35000.0,0.124432,15.96,1229.81,20,Tom and Holly Gores,3.0,0.142148,160000.0,0.160987,2012-01-01,NaN,0.254734,Small Business Loan,922xx,0.148025,4.90,0.0,136,720.0,724.0,1.0,9.0,23665.0,62.4,17.0,3.92953,1.81102,53.558127,0.0


In [4]:
y=df['loan_status']
df.drop(columns=['emp_title','issue_d','title','zip_code','loan_status'], inplace=True)

In [5]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3)

In [6]:
# Save desc column for natural language processing latter
desc_train=X_train['desc']
desc_test=X_test['desc']

X_train.drop(columns=['desc'], inplace=True)
X_test.drop(columns=['desc'], inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
# Standardize the data
scaler=StandardScaler()
scaler.fit(X_train)
scaled_train=scaler.transform(X_train)
scaled_test=scaler.transform(X_test)

In [8]:
lmt = WordNetLemmatizer()
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation and digits
    2. Remove all stopwords
    3. Lemmatization
    4. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    mess=str(mess)
    mess=mess[mess.find('>')+1:mess.find('<')]
    nopunc = [char for char in mess if (char not in string.punctuation) and (not char.isdigit())]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [lmt.lemmatize(word.lower()) for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [9]:
sentence=desc_train[0]
sentence

'  Borrower added on 12/31/13 > Bought a new house, furniture, water softener, a second car, etc. Got our lives started and now a manageable monthly payment will help keep them going!<br>'

In [10]:
text_process(sentence)

['bought',
 'new',
 'house',
 'furniture',
 'water',
 'softener',
 'second',
 'car',
 'etc',
 'got',
 'life',
 'started',
 'manageable',
 'monthly',
 'payment',
 'help',
 'keep',
 'going']

In [11]:
desc_train=desc_train.values.astype('U')
desc_test=desc_test.values.astype('U')

In [12]:
# vectorize all the words that have minimim frequency 0.001 and maximum frequency 0.9 in my training data
bow_transformer = CountVectorizer(analyzer=text_process, min_df=0.001, max_df=0.9).fit(desc_train)


In [13]:
# There are 2260 words in total
print(len(bow_transformer.vocabulary_))

528


In [14]:
bow_transformer.vocabulary_

{'na': 312,
 'want': 505,
 'pay': 337,
 'high': 213,
 'interest': 238,
 'rate': 375,
 'credit': 108,
 'card': 72,
 'im': 226,
 'applying': 30,
 'loan': 270,
 'consolidate': 98,
 'debt': 118,
 'ive': 247,
 'enough': 146,
 'recent': 385,
 'year': 526,
 'find': 170,
 'life': 263,
 'make': 284,
 'change': 78,
 'work': 520,
 'toward': 485,
 'better': 54,
 'future': 194,
 'paying': 339,
 'first': 172,
 'step': 453,
 'clear': 83,
 'payment': 340,
 'manage': 286,
 'expense': 157,
 'order': 329,
 'finally': 166,
 'get': 195,
 'forward': 181,
 'lender': 259,
 'able': 0,
 'one': 325,
 'month': 302,
 'working': 522,
 'towards': 486,
 'putting': 371,
 'financial': 168,
 'bill': 56,
 'paid': 333,
 'time': 479,
 'never': 315,
 'would': 524,
 'like': 264,
 'lower': 278,
 'build': 62,
 'history': 216,
 'also': 22,
 'sooner': 443,
 'recently': 386,
 'consolidation': 101,
 'k': 249,
 'within': 517,
 'period': 345,
 'cc': 76,
 'payoff': 341,
 'minimum': 298,
 'amount': 24,
 'instead': 235,
 'free': 184,
 

In [15]:
# Training data
messages_bow=bow_transformer.transform(desc_train)

In [16]:
messages_bow.shape

(126216, 528)

In [17]:
# Testing data
test_bow=bow_transformer.transform(desc_test)

In [18]:
test_bow.shape

(54093, 528)

In [19]:
# Fit a simple Multinomial Naive Bayes model on this bag of words
MNB = MultinomialNB()
MNB.fit(messages_bow, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
predictions=MNB.predict(test_bow)

In [21]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))


[[45499   156]
 [ 8385    53]]
              precision    recall  f1-score   support

         0.0       0.84      1.00      0.91     45655
         1.0       0.25      0.01      0.01      8438

    accuracy                           0.84     54093
   macro avg       0.55      0.50      0.46     54093
weighted avg       0.75      0.84      0.77     54093



In [22]:
MNB.score(test_bow, y_test)

0.8421052631578947

In [23]:
# Extract 100 principal components from the 2260 columns
from sklearn.decomposition import PCA
pca = PCA(n_components = 100)
pca.fit(messages_bow.todense())

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [24]:
pca.explained_variance_ratio_.sum()

0.7798980269040797

In [25]:
messages_bow_pca=pca.transform(messages_bow.todense())

In [26]:
messages_bow_pca

array([[-7.25817417e-01, -6.44506626e-02,  3.92857214e-02, ...,
        -1.03102725e-04,  6.55102827e-05,  5.08892437e-07],
       [-7.25817417e-01, -6.44506626e-02,  3.92857214e-02, ...,
        -1.03102725e-04,  6.55102827e-05,  5.08892437e-07],
       [-7.25817417e-01, -6.44506626e-02,  3.92857214e-02, ...,
        -1.03102725e-04,  6.55102827e-05,  5.08892437e-07],
       ...,
       [-7.25817417e-01, -6.44506626e-02,  3.92857214e-02, ...,
        -1.03102725e-04,  6.55102827e-05,  5.08892437e-07],
       [-7.25817417e-01, -6.44506626e-02,  3.92857214e-02, ...,
        -1.03102725e-04,  6.55102827e-05,  5.08892437e-07],
       [ 1.48703744e+00,  5.13897498e-02,  3.44491021e-01, ...,
        -6.01338419e-02,  5.87004781e-02, -1.31172081e-01]])

In [27]:
test_bow_pca=pca.transform(test_bow.todense())

In [28]:
test_bow_pca.shape

(54093, 100)

In [29]:
df1=pd.DataFrame(messages_bow_pca)

In [30]:
# This is the original dataset with all the numeric variables
df2=pd.DataFrame(scaled_train)

In [31]:
# Combine the 100 principal components from NLP with the other numeric variables
df=pd.concat([df1,df2], axis=1)

In [32]:
# Do the same thing for the test set
test_df=pd.concat([pd.DataFrame(test_bow_pca), pd.DataFrame(scaled_test)], axis=1)

In [33]:
# Fit a logistic regression model on this combination of numeric variables and the 'desc' column
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(df,y_train)
predictions = logmodel.predict(test_df)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))
print(logmodel.score(test_df, y_test))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

         0.0       0.85      1.00      0.91     45655
         1.0       0.46      0.01      0.03      8438

    accuracy                           0.84     54093
   macro avg       0.65      0.51      0.47     54093
weighted avg       0.79      0.84      0.78     54093

[[45511   144]
 [ 8316   122]]
0.8436026842659864


In [34]:
# Grid search for the best combination of hyperparameters
from sklearn.model_selection import GridSearchCV
param_grid={'penalty': ['l1','l2','none'], 'C':[0.01,0.1,1,5,10,50,100,500],
            'class_weight': [None, 'balanced'], 'solver': ['liblinear','newton-cg','lbfgs','sag','saga']}


In [35]:
grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, error_score=0, verbose=3)
grid.fit(df,y_train)

Fitting 3 folds for each of 240 candidates, totalling 720 fits
[CV] C=0.01, class_weight=None, penalty=l1, solver=liblinear .........


/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.01, class_weight=None, penalty=l1, solver=liblinear, score=0.845, total=   1.1s
[CV] C=0.01, class_weight=None, penalty=l1, solver=liblinear .........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV]  C=0.01, class_weight=None, penalty=l1, solver=liblinear, score=0.844, total=   1.5s
[CV] C=0.01, class_weight=None, penalty=l1, solver=liblinear .........


...

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


[CV]  C=500, class_weight=balanced, penalty=none, solver=saga, score=0.642, total=  13.0s
[CV] C=500, class_weight=balanced, penalty=none, solver=saga .........


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed: 78.5min finished


[CV]  C=500, class_weight=balanced, penalty=none, solver=saga, score=0.644, total=  13.4s


GridSearchCV(cv='warn', error_score=0,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1, 5, 10, 50, 100, 500],
                         'class_weight': [None, 'balanced'],
                         'penalty': ['l1', 'l2', 'none'],
                         'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=Tr

In [36]:
grid.best_params_

{'C': 0.01, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'}

In [37]:
grid_predictions = grid.predict(test_df)
print(classification_report(y_test,grid_predictions))
print(confusion_matrix(y_test,grid_predictions))
print(grid.score(test_df, y_test))

              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92     45655
         1.0       0.48      0.01      0.02      8438

    accuracy                           0.84     54093
   macro avg       0.66      0.50      0.47     54093
weighted avg       0.79      0.84      0.78     54093

[[45555   100]
 [ 8345    93]]
0.8438799844711885


In [38]:
# The result improved a little bit from the original logistic regression which only uses numeric variables (grid.score is 0.8436026842659864)
